In [4]:
# Notebook Initialization

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # Data visualization
from matplotlib.ticker import MaxNLocator
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats

import scipy.stats
from sklearn.decomposition import PCA # Principal Component Analysis
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
    facecolor = '#f0f3f4',
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

%config InlineBackend.figure_format = 'retina'

C:\Users\user\anaconda3\envs\R-LAB\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# <div style="color:white;display:fill;border-radius:5px;background-color:#38c9ff;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:#FFFFFF;overflow:hidden;margin:0;font-size:110%"><b>1 |</b>  Data Overview</p></div>

## <b><span style='color:#38c9ff'>1.1</span> | Main Goal</b>
In this competition, you'll classify 60-second sequences of sensor data, indicating whether a subject was in either of two activity states for the duration of the sequence.

## <b><span style='color:#38c9ff'>1.2</span> | Files and Field Descriptions</b>

1. **train.csv** - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
    - `sequence` - a unique id for each sequence
    - `subject` - a unique id for the subject in the experiment
    - `step` - time step of the recording, in one second intervals
    - `sensor_00 - sensor_12` - the value for each of the thirteen sensors at that time step
2. **train_labels.csv** - the class label for each sequence.
    - `sequence` - the unique id for each sequence.
    - `state` - the state associated to each sequence. This is the target which you are trying to predict.
3. **test.csv** - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')

print("Train Dataset -------------------------------------------------")
print("There are {:,} observations and {} columns in the train dataset.".format(train.shape[0], train.shape[1]))
print("There are {} missing values in the train dataset.".format(train.isna().sum().sum()))
print("There are {} duplicated values the train dataset.".format(train.duplicated().sum()))
print("")
print("Labels Dataset ------------------------------------------------")
print("There are {:,} observations and {} columns in the labels dataset.".format(train_labels.shape[0], train_labels.shape[1]))
print("There are {} missing values in the labels dataset.".format(train_labels.isna().sum().sum()))
print("There are {} duplicated values the labels dataset.".format(train_labels.duplicated().sum()))
print("")
print("Test Dataset --------------------------------------------------")
print("There are {:,} observations and {} columns in the test dataset.".format(train_labels.shape[0], train_labels.shape[1]))
print("There are {} missing values in the test dataset.".format(train_labels.isna().sum().sum()))
print("There are {} duplicated values the test dataset.".format(train_labels.duplicated().sum()))

<div class="alert alert-block alert-info" style="color:#03162D;border-color:#38c9ff;background-color:#d2f2ff; font-size:12px; font-family:verdana; line-height: 1.7em;">
    <b><span style='color:#38c9ff'>Observations</span></b> | The Train and Labels Datasets

- There are **25968 Secuences** based on the `labels` dataset, and the `train` dataset has the 60 steps of each secuence, so there are **1558080 records**.

- Every sequence has **13 values** (`sensor_00 - sensor_12`) and **60 steps**, so there are **780 features** in this binary classification problem
    

    

In [ ]:
display(train.describe(include=['float','int']).T.round(3).style.format('{:,.2f}')
        .bar(color='#38c9ff', axis=0, vmin=0)
        .set_caption("Summary statistics for Train Dataset"))

In [ ]:
display(train_labels.describe(include=['float','int']).T.round(3).style.format('{:,.2f}')
        .bar(color='#38c9ff', axis=0, vmin=0)
        .set_caption("Summary statistics for Train Labels Dataset"))

# <div style="color:white;display:fill;border-radius:5px;background-color:#38c9ff;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:#FFFFFF;overflow:hidden;margin:0;font-size:110%"><b>2 |</b>  Exploratory Data Analysis</p></div>

## <b><span style='color:#38c9ff'>2.1</span> | The Subjects</b>

In [ ]:
plt.subplots(1, 2, sharey=True, figsize=(16, 4))

def plot_sequence_count_distribution(df, title):
    temp = df.subject.value_counts().sort_values() // 60
    plt.bar(range(len(temp)), temp, width=1, color = '#38c9ff')
    plt.xlabel('subject')
    plt.ylabel('sequence count')
    plt.title(f'Sequence count distribution over {title} subjects')
    display(temp.sort_values().rename(f'sequence count per {title} subject'))

plt.subplot(1, 2, 1)
plot_sequence_count_distribution(train, 'training')
plt.subplot(1, 2, 2)
plot_sequence_count_distribution(test, 'test')
plt.show()

In [ ]:
temp = train.groupby('sequence').subject.min() # dataframe with one row per sequence
temp = train_labels.merge(temp, on='sequence') # adding the labels
temp = temp.groupby('subject').agg({'state': 'mean', 'sequence': 'count'}).rename(columns={'state': 'probability', 'sequence': 'sequence_count'})
temp1 = temp[temp.sequence_count >= 25].probability.rename('Probability of state==1')

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.hist(temp1, bins=20,color = '#38c9ff')
plt.ylabel('Subject count')
plt.xlabel('Probability for state==1')
plt.title('Histogram of state probabilities per subject')

plt.subplot(1, 2, 2)
plt.scatter(temp.sequence_count, temp.probability,color = '#38c9ff')
plt.xlabel('sequence count')
plt.ylabel('probability')
plt.title('Probability depends on sequence count')

plt.show()

print()
print(f"The standard deviation of {temp[temp.sequence_count >= 25].probability.std():.2f} is much higher than 0.1.")
print()
print('Subjects which are always in state 0:', (temp.probability == 0).sum())

## <b><span style='color:#38c9ff'>2.2</span> | The Sensors</b>

In [ ]:
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    plt.hist(train[sensor_name], bins=100, color = '#38c9ff')
    plt.title(f"{sensor_name} histogram")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Histograms Before Outlier Removal', y=1.02)
plt.show()

In [ ]:
figure = plt.figure(figsize=(16, 8))
for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    plt.subplot(4, 4, sensor+1)
    plt.hist(train[sensor_name], bins=100, color = '#38c9ff',
             range=(train[sensor_name].quantile(0.02),
                    train[sensor_name].quantile(0.98)))
    plt.title(f"{sensor_name} histogram")
figure.tight_layout(h_pad=1.0, w_pad=0.5)
plt.suptitle('Sensor Histograms After Outlier Removal', y=1.02)
plt.show()

In [ ]:
sensor_name = 'sensor_12'
plt.hist(train[sensor_name], bins=100, color='#38c9ff',
         range=(train[sensor_name].quantile(0.15),
                train[sensor_name].quantile(0.85)))
plt.show()

## <b><span style='color:#38c9ff'>2.3</span> | Time Series</b>

In [ ]:
sequences = [0, 1, 2, 8364, 15404]
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16))
for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        plt.plot(range(60), train[train.sequence == sequence][sensor_name],
                color=plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}")
        if sequence == sequences[0]: plt.ylabel(sensor_name)
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02)
plt.show()

In [ ]:
# For every sensor: count the sequences where the sensor is stuck at a constant value
def stuck_at_constant(seq):
    return seq.min() == seq.max()

for sensor in range(13):
    sensor_name = f"sensor_{sensor:02d}"
    stuck_sequences = train.groupby('sequence')[sensor_name].apply(stuck_at_constant)
    print(f"{sensor_name}: {stuck_sequences.sum():4d}   {train_labels[stuck_sequences].state.mean()}")

## <b><span style='color:#38c9ff'>2.4</span> | Pivoting the Data</b>

In [ ]:
train_pivoted = train.pivot(index=['sequence', 'subject'], columns='step', values=[col for col in train.columns if 'sensor_' in col])
train_pivoted

In [ ]:
temp = train_pivoted.sort_values(by=list(train_pivoted.columns))
duplicates_first = temp.duplicated(keep='first')
duplicates_last = temp.duplicated(keep='last')
temp['duplicates_first'] = duplicates_first
temp['duplicates_last'] = duplicates_last
duplicates = temp[duplicates_first | duplicates_last]
display(duplicates)

print()
print('All these sequences have sensor_00 stuck at 0.000773:', duplicates['sensor_00'].apply(stuck_at_constant).all())

print()
print(f'Labels of the duplicates: {list(train_labels.loc[duplicates.index.get_level_values(0)].state)}')

## <b><span style='color:#38c9ff'>2.5</span> | PCA</b>

In [ ]:
# Compute the PCA
# Outlier removal or input scaling may change the PCA completely
# whiten=True/False doesn't change the look of the diagrams (it only modifies the scale)
def plot_pca(df, col, title):
    """Plot cumulative variance and the first two components in column col of the figure."""
    pca = PCA()
    #pca.fit(StandardScaler().fit_transform(train_df.drop(columns=['id', 'target'])))
    Xt = pca.fit_transform(df.values)

    # Plot the cumulative explained variance
    plt.subplot(2, 2, col+1)
    plt.plot(np.cumsum(pca.explained_variance_ratio_), color='#38c9ff')
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance')
    plt.title(title)

    # Scatterplot of the first two dimensions
    plt.subplot(2, 2, col+3)
    plt.scatter(Xt[0], Xt[1], color='#38c9ff')
    
temp = train_pivoted.clip(train_pivoted.quantile(0.02, axis=0).values,
                          train_pivoted.quantile(0.98, axis=0).values, 
                          axis=1)
temp.pop('sensor_12')

plt.figure(figsize=(12, 8))
plot_pca(train_pivoted, 0, 'Before Outlier Removal')
plot_pca(temp, 1, 'After Outlier Removal')
plt.suptitle('Principal Components Analysis')
plt.tight_layout(h_pad=1.1)
plt.show()

# <div style="color:white;display:fill;border-radius:5px;background-color:#38c9ff;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:#FFFFFF;overflow:hidden;margin:0;font-size:110%"><b>3 |</b>  Modeling</p></div>

## <b><span style='color:#38c9ff'>3.1</span> | Feature Engineering</b>

In [ ]:
sensors = [col for col in train.columns if 'sensor_' in col]

train_pivoted0 = train.pivot(index=['sequence', 'subject'], columns='step', values=sensors)
display(train_pivoted0)

In [ ]:
# Feature engineering
def engineer(df):
    new_df = pd.DataFrame([], index=df.index)
    for sensor in sensors:
        new_df[sensor + '_mean'] = df[sensor].mean(axis=1)
        new_df[sensor + '_std'] = df[sensor].std(axis=1)
        new_df[sensor + '_iqr'] = scipy.stats.iqr(df[sensor], axis=1)
        new_df[sensor + '_sm'] = np.nan_to_num(new_df[sensor + '_std'] / 
                                               new_df[sensor + '_mean'].abs()).clip(-1e30, 1e30)
        new_df[sensor + '_kurtosis'] = scipy.stats.kurtosis(df[sensor], axis=1)
    new_df['sensor_02_up'] = (df.sensor_02.diff(axis=1) > 0).sum(axis=1)
    new_df['sensor_02_down'] = (df.sensor_02.diff(axis=1) < 0).sum(axis=1)
    new_df['sensor_02_upsum'] = df.sensor_02.diff(axis=1).clip(0, None).sum(axis=1)
    new_df['sensor_02_downsum'] = df.sensor_02.diff(axis=1) .clip(None, 0).sum(axis=1)
    new_df['sensor_02_upmax'] = df.sensor_02.diff(axis=1).max(axis=1)
    new_df['sensor_02_downmax'] = df.sensor_02.diff(axis=1).min(axis=1)
    new_df['sensor_02_upmean'] = np.nan_to_num(new_df['sensor_02_upsum'] / new_df['sensor_02_up'], posinf=40)
    new_df['sensor_02_downmean'] = np.nan_to_num(new_df['sensor_02_downsum'] / new_df['sensor_02_down'], neginf=-40)
    return new_df

train_pivoted = engineer(train_pivoted0)

train_shuffled = train_pivoted.sample(frac=1.0, random_state=1)
labels_shuffled = train_labels.reindex(train_shuffled.index.get_level_values('sequence'))
labels_shuffled = labels_shuffled[['state']].merge(train[['sequence', 'subject']].groupby('sequence').min(),
                                                   how='left', on='sequence')
labels_shuffled = labels_shuffled.merge(labels_shuffled.groupby('subject').size().rename('sequence_count'),
                                        how='left', on='subject')
train_shuffled['sequence_count_of_subject'] = labels_shuffled['sequence_count'].values

selected_columns = train_shuffled.columns
print(len(selected_columns))
#train_shuffled.columns

In [ ]:
# Plot dependence between every feature and the target
ncols = len(train_shuffled.columns) // 13
plt.subplots(15, ncols, sharey=True, sharex=True, figsize=(15, 40))
for i, col in enumerate(train_shuffled.columns):
    temp = pd.DataFrame({col: train_shuffled[col].values,
                         'state': labels_shuffled.state.values})
    temp = temp.sort_values(col)
    temp.reset_index(inplace=True)
    plt.subplot(15, ncols, i+1)
    plt.scatter(temp.index, temp.state.rolling(1000).mean(), s=2,color='#38c9ff')
    plt.xlabel(col)
    plt.xticks([])
plt.show()

## <b><span style='color:#38c9ff'>3.2</span> | Feature Selection</b>

In [ ]:
# Drop some useless features
dropped_features = ['sensor_05_kurtosis', 'sensor_08_mean',
                    'sensor_05_std', 'sensor_06_kurtosis',
                    'sensor_06_std', 'sensor_03_std',
                    'sensor_02_kurtosis', 'sensor_03_kurtosis',
                    'sensor_09_kurtosis', 'sensor_03_mean',
                    'sensor_00_mean', 'sensor_02_iqr',
                    'sensor_05_mean', 'sensor_06_mean',
                    'sensor_07_std', 'sensor_10_iqr',
                    'sensor_11_iqr', 'sensor_12_iqr',
                    'sensor_09_mean', 'sensor_02_sm',
                    'sensor_03_sm', 'sensor_05_iqr', 
                    'sensor_06_sm', 'sensor_09_iqr', 
                    'sensor_07_iqr', 'sensor_10_mean']
selected_columns = [f for f in selected_columns if f not in dropped_features]
len(selected_columns)

In [ ]:
# Sequential feature selection
# This code is a more verbose form of scikit-learn's SequentialFeatureSelector
estimator = HistGradientBoostingClassifier(learning_rate=0.05, max_leaf_nodes=25,
                                       max_iter=1000, min_samples_leaf=500,
                                       l2_regularization=1,
                                       max_bins=255,
                                       random_state=4, verbose=0)

X, y = train_shuffled[selected_columns], labels_shuffled.state
n_iterations, backward = 48, False

if n_iterations != 0:
    n_features = X.shape[1]
    current_mask = np.zeros(shape=n_features, dtype=bool)
    history = []
    for _ in range(n_iterations):
        candidate_feature_indices = np.flatnonzero(~current_mask)
        scores = {}
        for feature_idx in candidate_feature_indices:
            candidate_mask = current_mask.copy()
            candidate_mask[feature_idx] = True
            X_new = X.values[:, ~candidate_mask if backward else candidate_mask]
            scores[feature_idx] = cross_val_score(
                estimator,
                X_new,
                y,
                cv=GroupKFold(n_splits=5),
                groups=train_shuffled.index.get_level_values('subject'),
                scoring='roc_auc',
                n_jobs=-1,
            ).mean()
            #print(f"{str(X.columns[feature_idx]):30} {scores[feature_idx]:.3f}")
        new_feature_idx = max(scores, key=lambda feature_idx: scores[feature_idx])
        current_mask[new_feature_idx] = True
        history.append(scores[new_feature_idx])
        new = 'Deleted' if backward else 'Added'
        print(f'{new} feature: {str(X.columns[new_feature_idx]):30}'
              f' {scores[new_feature_idx]:.3f}')

    print()
    plt.figure(figsize=(12, 6))
    plt.scatter(np.arange(len(history)) + (0 if backward else 1), history, color='#38c9ff')
    plt.ylabel('AUC')
    plt.xlabel('Features removed' if backward else 'Features added')
    plt.title('Sequential Feature Selection')
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.show()

    if backward:
        current_mask = ~current_mask
    selected_columns = np.array(selected_columns)[current_mask]
    print(selected_columns)

## <b><span style='color:#38c9ff'>3.3</span> | Cross Validation</b>

In [ ]:
%%time
# Cross-validation of the classifier

print(f"{len(selected_columns)} features")
score_list = []
kf = GroupKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_shuffled, groups=train_shuffled.index.get_level_values('subject'))):
    X_tr = train_shuffled.iloc[idx_tr][selected_columns]
    X_va = train_shuffled.iloc[idx_va][selected_columns]
    y_tr = labels_shuffled.iloc[idx_tr].state
    y_va = labels_shuffled.iloc[idx_va].state

    model = HistGradientBoostingClassifier(learning_rate=0.05, max_leaf_nodes=25,
                                           max_iter=1000, min_samples_leaf=500,
                                           l2_regularization=1,
                                           validation_fraction=0.05,
                                           max_bins=63,
                                           random_state=3, verbose=0)
#     model = XGBClassifier(n_estimators=500, n_jobs=-1,
#                           eval_metric=['logloss'],
#                           #max_depth=10,
#                           colsample_bytree=0.8,
#                           #gamma=1.4,
#                           reg_alpha=6, reg_lambda=1.5,
#                           tree_method='hist',
#                           learning_rate=0.03,
#                           verbosity=1,
#                           use_label_encoder=False, random_state=3)

    if True or type(model) != XGBClassifier:
        model.fit(X_tr.values, y_tr)
    else:
        model.fit(X_tr.values, y_tr, eval_set = [(X_va.values, y_va)], 
                  eval_metric = ['auc'], early_stopping_rounds=30, verbose=10)
    try:
        y_va_pred = model.decision_function(X_va.values) # HistGradientBoostingClassifier
    except AttributeError:
        try:
            y_va_pred = model.predict_proba(X_va.values)[:,1] # XGBClassifier
        except AttributeError:
            y_va_pred = model.predict(X_va.values) # XGBRegressor
    score = roc_auc_score(y_va, y_va_pred)
    try:
        print(f"Fold {fold}: n_iter ={model.n_iter_:5d}    AUC = {score:.3f}")
    except AttributeError:
        print(f"Fold {fold}:                  AUC = {score:.3f}")
    score_list.append(score)
    
print(f"OOF AUC:                       {np.mean(score_list):.3f}") # 0.944

## <b><span style='color:#38c9ff'>3.4</span> | ROC Curve</b>

In [ ]:
# Plot the roc curve for the last fold
def plot_roc_curve(y_va, y_va_pred):
    plt.figure(figsize=(8, 8))
    fpr, tpr, _ = roc_curve(y_va, y_va_pred)
    plt.plot(fpr, tpr, color='r', lw=2,color='#38c9ff')
    plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--")
    plt.gca().set_aspect('equal')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic")
    plt.show()

plot_roc_curve(y_va, y_va_pred)

## <b><span style='color:#38c9ff'>3.5</span> | Test predictions and submission</b>

In [ ]:
# Feature engineering for test
test_pivoted0 = test.pivot(index=['sequence', 'subject'], columns='step', values=sensors)
test_pivoted = engineer(test_pivoted0)
sequence_count = test_pivoted.index.to_frame(index=False).groupby('subject').size().rename('sequence_count_of_subject')
#display(test_pivoted.head(2))
submission = pd.DataFrame({'sequence': test_pivoted.index.get_level_values('sequence')})
test_pivoted = test_pivoted.merge(sequence_count, how='left', on='subject')
test_pivoted.head(2)

In [ ]:
# Retrain, predict and write submission
print(f"{len(selected_columns)} features")

pred_list = []
for seed in range(100):
    X_tr = train_shuffled[selected_columns]
    y_tr = labels_shuffled.state

    model = HistGradientBoostingClassifier(learning_rate=0.05, max_leaf_nodes=25,
                                           max_iter=1000, min_samples_leaf=500,
                                           validation_fraction=0.05,
                                           l2_regularization=1,
                                           max_bins=63,
                                           random_state=seed, verbose=0)
    model.fit(X_tr.values, y_tr)
    pred_list.append(scipy.stats.rankdata(model.decision_function(test_pivoted[selected_columns].values)))
    print(f"{seed:2}", pred_list[-1])
print()
submission['state'] = sum(pred_list) / len(pred_list)
submission.to_csv('submission.csv', index=False)
submission